In [ ]:
from langchain.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
    SystemMessagePromptTemplate,
)

generator_prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessagePromptTemplate.from_template(
            """You are a helpful assistant. Answer this question as good as you can. You are being revised.
            If you are being revised, just reformulate your original answer. Original Question:"""
        ),
        HumanMessagePromptTemplate.from_template("{query}"),
        MessagesPlaceholder("revision_dialogue"),
        SystemMessagePromptTemplate.from_template(
            "If the revisor asked you to reformulate your answer, please do so. Do NOT apologize. JUST REFORMULATE."
        ),
    ]
)

revisor_prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessagePromptTemplate.from_template(
            """You are a Revisor. You are revising the answer of a chatbot. Your principles are as follows:
                {principles}
                You can either return the answer from the chatbot to the user or tell the chatbot to reformulate his answer if any principles are violated.
                Original Question:"""
        ),
        HumanMessagePromptTemplate.from_template("{query}"),
        MessagesPlaceholder("revision_dialogue"),
    ]
)

In [ ]:
import operator
from typing import Annotated, Optional, Sequence, TypedDict

from langchain_core.messages import BaseMessage


class AgentState(TypedDict):
    query: str
    principles: str
    revision_dialogue: Annotated[Sequence[BaseMessage], operator.add]
    answer: Optional[str]

In [ ]:
from helpers import llm

model = llm(temperature=0)

In [ ]:
from langchain_core._api import suppress_langchain_beta_warning
from langchain_core.messages import HumanMessage
from langchain_core.pydantic_v1 import BaseModel, Field


def generator(state):
    generator_chain = generator_prompt | model

    proposal = generator_chain.invoke(
        {"query": state["query"], "revision_dialogue": state["revision_dialogue"] or []}
    )

    return {"revision_dialogue": [proposal]}


def revisor(state):
    class Revision(BaseModel):
        acceptable: str = Field(
            description="Is the answer of the chatbot compliant with the principles? answer is 'yes' or 'no'"
        )
        critique: Optional[str] = Field(
            description="Critique of the document. Leave out if answer is acceptable.",
            default=None,
        )

    with suppress_langchain_beta_warning():
        revisor_model = model.with_structured_output(Revision)

    revisor_chain = revisor_prompt | revisor_model

    revision: Revision = revisor_chain.invoke(
        {
            "query": state["query"],
            "revision_dialogue": state["revision_dialogue"],
            "principles": state["principles"],
        }
    )

    accepted = revision.acceptable == "yes" and revision.critique is None

    if accepted:
        return {"answer": state["revision_dialogue"][-1].content}
    else:
        return {"revision_dialogue": [HumanMessage(content=revision.critique)]}


def is_accepted(state):
    if state["answer"] is None:
        return "rejected"
    else:
        return "accepted"

In [ ]:
from langgraph.graph import END, StateGraph

workflow = StateGraph(AgentState)
workflow.add_node("generator", generator)
workflow.add_node("revisor", revisor)
workflow.set_entry_point("generator")
workflow.add_edge("generator", "revisor")
workflow.add_conditional_edges(
    "revisor",
    is_accepted,
    {
        "accepted": END,
        "rejected": "generator",
    },
)

app = workflow.compile()

In [ ]:
inputs = {
    "query": "Tell me a joke about star trek (or any other sci-fi series).",
    "principles": "The answer must not contain any jokes about star trek.",
}

In [ ]:
for chunk in app.stream(input=inputs):
    for k, v in chunk.items():
        print(f"answer from node: {k}")
        print(v)
        if k == "__end__":
            print("\n--- Final answer ---\n")
            print(v["answer"])